In [1]:
import lovely_tensors as lt
from lovely_numpy import lovely, set_config
from rich import print

lt.monkey_patch()
set_config(repr=lovely)
print("Loaded")

Loaded

In [13]:
from ll.actsave import ActivationLoader

escn = ActivationLoader.from_latest_version("/mnt/shared/ocp-actsave-2/escn/")
# print(escn)

rhescn = ActivationLoader.from_latest_version(
    "/mnt/shared/ocp-actsave-2/rhescnv8clean/"
)
# print(rhescn)


def toplevel_keys(activations):
    return {k: v.num_activations for k, v in activations.items() if k.count(".") <= 1}


print(
    {
        "escn": toplevel_keys(escn.activations),
        "rhescn": toplevel_keys(rhescn.activations),
    }
)

{
    'escn': {
        'wigner': 1,
        'wigner_inv': 1,
        'x_embedding_l0m0': 1,
        'x_embedding': 1,
        'LayerBlock_0.x': 1,
        'LayerBlock_0.x_message': 2,
        'LayerBlock_0.x_message_updated': 1,
        'LayerBlock_0.x_updated': 1,
        'LayerBlock_1.x': 1,
        'LayerBlock_1.x_updated': 1,
        'LayerBlock_1.x_message': 2,
        'LayerBlock_1.x_message_updated': 1,
        'LayerBlock_2.x': 1,
        'LayerBlock_2.x_message': 2,
        'LayerBlock_2.x_updated': 1,
        'LayerBlock_3.x': 1,
        'LayerBlock_2.x_message_updated': 1,
        'LayerBlock_3.x_updated': 1,
        'LayerBlock_3.x_message_updated': 1,
        'LayerBlock_3.x_message': 2,
        'LayerBlock_4.x': 1,
        'LayerBlock_4.x_message_updated': 1,
        'LayerBlock_4.x_updated': 1,
        'LayerBlock_5.x': 1,
        'LayerBlock_4.x_message': 2,
        'LayerBlock_5.x_message_updated': 1,
        'LayerBlock_5.x_message': 2,
        'LayerBlock_5.x_updated': 1,
        'LayerBlock_6.x': 1,
        'LayerBlock_6.x_updated': 1,
        'LayerBlock_6.x_message': 2,
        'LayerBlock_6.x_message_updated': 1,
        'LayerBlock_7.x': 1,
        'LayerBlock_7.x_message_updated': 1,
        'LayerBlock_7.x_message': 2,
        'LayerBlock_7.x_updated': 1,
        'LayerBlock_8.x': 1,
        'LayerBlock_8.x_message_updated': 1,
        'LayerBlock_8.x_updated': 1,
        'LayerBlock_9.x': 1,
        'LayerBlock_8.x_message': 2,
        'LayerBlock_9.x_message_updated': 1,
        'LayerBlock_9.x_message': 2,
        'LayerBlock_9.x_updated': 1,
        'LayerBlock_10.x': 1,
        'LayerBlock_10.x_message_updated': 1,
        'LayerBlock_10.x_message': 2,
        'LayerBlock_10.x_updated': 1,
        'LayerBlock_11.x': 1,
        'LayerBlock_11.x_message': 2,
        'LayerBlock_11.x_message_updated': 1,
        'x_pt': 1,
        'LayerBlock_11.x_updated': 1,
        'node_energy': 1,
        'forces': 1,
        'energy': 1
    },
    'rhescn': {
        'full_wigner': 1,
        'wigner_tri_to_rh': 1,
        'wigner_inv_from_grid': 1,
        'M0L0Embedding.x_l0m0': 1,
        'M0L0Embedding.x': 1,
        'LayerBlock_0.x': 1,
        'LayerBlock_0.x_message': 1,
        'LayerBlock_1.x': 1,
        'LayerBlock_1.x_message': 1,
        'LayerBlock_2.x': 1,
        'LayerBlock_2.x_message': 1,
        'LayerBlock_3.x': 1,
        'LayerBlock_3.x_message': 1,
        'LayerBlock_4.x': 1,
        'LayerBlock_4.x_message': 1,
        'LayerBlock_5.x': 1,
        'LayerBlock_5.x_message': 1,
        'LayerBlock_6.x': 1,
        'LayerBlock_6.x_message': 1,
        'LayerBlock_7.x': 1,
        'LayerBlock_7.x_message': 1,
        'LayerBlock_8.x': 1,
        'LayerBlock_8.x_message': 1,
        'LayerBlock_9.x': 1,
        'LayerBlock_9.x_message': 1,
        'LayerBlock_10.x': 1,
        'LayerBlock_10.x_message': 1,
        'LayerBlock_11.x': 1,
        'LayerBlock_11.x_message': 1,
        'x_pt': 1,
        'node_energy': 1,
        'forces': 1,
        'energy': 1
    }
}

In [3]:
import numpy as np
from bidict import bidict
from einops import rearrange
from IPython.display import Markdown, display

LMAX = 4
MMAX = 2
RH_MMAX = 4


def trimask(x: np.ndarray, lmax: int = LMAX):
    return x[:, : (lmax + 1) ** 2]


def _escnindices(lmax: int, mmax: int | None):
    coeffs = bidict[tuple[int, int], int]()
    for l in range(lmax + 1):
        for m in range(-l, l + 1):
            if mmax is not None and abs(m) > mmax:
                continue

            coeffs[(l, m)] = len(coeffs)
    return coeffs


def trimask_mmax(
    x: np.ndarray,
    lmax: int = LMAX,
    mmax: int = MMAX,
):
    indices: list[int] = []
    i = 0
    for l in range(lmax + 1):
        for m in range(-l, l + 1):
            if abs(m) > mmax:
                i += 1
                continue

            indices.append(i)
            i += 1

    return x[:, indices]


def rhmask(
    x: np.ndarray,
    lmax: int = LMAX,
    mmax: int | None = MMAX,
    rh_mmax: int = RH_MMAX,
):
    if x.ndim == 3:
        x = rearrange(
            x,
            "E (m two_sign l) C -> E m two_sign l C",
            m=rh_mmax + 1,
            two_sign=2,
        )

    idx = _escnindices(lmax, mmax)
    x_masked = np.zeros((x.shape[0], len(idx), *x.shape[4:]), dtype=x.dtype)
    for (l, m), i in idx.items():
        signidx = int(np.signbit(m).item())
        x_masked[:, i] = x[:, abs(m), signidx, l - abs(m)]
    return x_masked

In [4]:
def check_layer(layer_idx: int):
    # region  x_edge
    display(Markdown(f"## Layer {layer_idx} x_edge"))
    print(
        {
            "rhescn": (
                x_rh := rhescn.activations[
                    f"LayerBlock_{layer_idx}.MessageBlock.x_edge"
                ][0]
            ),
            "escn": (
                x_escn := escn.activations[
                    f"LayerBlock_{layer_idx}.message_block.x_edge"
                ][0]
            ),
        },
    )
    np.testing.assert_allclose(x_rh, x_escn, atol=1.0e-4)
    # endregion

    # region  x_source
    display(Markdown(f"## Layer {layer_idx} x_source"))
    print(
        {
            "rhescn": (
                x_rh := trimask(
                    rhescn.activations[f"LayerBlock_{layer_idx}.MessageBlock.x_source"][
                        0
                    ]
                )
            ),
            "escn": (
                x_escn := escn.activations[
                    f"LayerBlock_{layer_idx}.message_block.x_source"
                ][0]
            ),
        },
    )
    np.testing.assert_allclose(x_rh, x_escn, atol=1.0e-4)
    # endregion

    # region  x_target
    display(Markdown(f"## Layer {layer_idx} x_target"))
    print(
        {
            "rhescn": (
                x_rh := trimask(
                    rhescn.activations[f"LayerBlock_{layer_idx}.MessageBlock.x_target"][
                        0
                    ]
                )
            ),
            "escn": (
                x_escn := escn.activations[
                    f"LayerBlock_{layer_idx}.message_block.x_target"
                ][0]
            ),
        },
    )
    np.testing.assert_allclose(x_rh, x_escn, atol=1.0e-4)
    # endregion

    # region  x_source_rot
    display(Markdown(f"## Layer {layer_idx} x_source_rot"))
    print(
        {
            "rhescn": (
                x_rh := rhmask(
                    rhescn.activations[
                        f"LayerBlock_{layer_idx}.MessageBlock.x_source_rot"
                    ][0]
                )
            ),
            "escn": (
                x_escn := escn.activations[
                    f"LayerBlock_{layer_idx}.message_block.x_source_rot"
                ][0]
            ),
        },
    )
    np.testing.assert_allclose(x_rh, x_escn, atol=1.0e-4)
    # endregion

    # region  x_target_rot
    display(Markdown(f"## Layer {layer_idx} x_target_rot"))
    print(
        {
            "rhescn": (
                x_rh := rhmask(
                    rhescn.activations[
                        f"LayerBlock_{layer_idx}.MessageBlock.x_target_rot"
                    ][0]
                )
            ),
            "escn": (
                x_escn := escn.activations[
                    f"LayerBlock_{layer_idx}.message_block.x_target_rot"
                ][0]
            ),
        },
    )
    np.testing.assert_allclose(x_rh, x_escn, atol=1.0e-4)
    # endregion

    # region  x_source_updated
    display(Markdown(f"## Layer {layer_idx} x_source_updated"))
    print(
        {
            "rhescn": (
                x_rh := rhmask(
                    rhescn.activations[
                        f"LayerBlock_{layer_idx}.MessageBlock.so2_block_source.x_source_updated"
                    ][0]
                )
            ),
            "escn": (
                x_escn := escn.activations[
                    f"LayerBlock_{layer_idx}.message_block.x_source_post_so2"
                ][0]
            ),
        },
    )
    np.testing.assert_allclose(x_rh, x_escn, atol=1.0e-4)
    # endregion

    # region  x_target_updated
    display(Markdown(f"## Layer {layer_idx} x_target_updated"))
    print(
        {
            "rhescn": (
                x_rh := rhmask(
                    rhescn.activations[
                        f"LayerBlock_{layer_idx}.MessageBlock.so2_block_target.x_target_updated"
                    ][0]
                )
            ),
            "escn": (
                x_escn := escn.activations[
                    f"LayerBlock_{layer_idx}.message_block.x_target_post_so2"
                ][0]
            ),
        },
    )
    np.testing.assert_allclose(x_rh, x_escn, atol=1.0e-4)
    # endregion

    # region  x_updated
    display(Markdown(f"## Layer {layer_idx} x_updated"))
    print(
        {
            "rhescn": (
                x_rh := rhmask(
                    rhescn.activations[
                        f"LayerBlock_{layer_idx}.MessageBlock.x_updated"
                    ][0]
                )
            ),
            "escn": (
                x_escn := escn.activations[
                    f"LayerBlock_{layer_idx}.message_block.x_updated"
                ][0]
            ),
        },
    )
    np.testing.assert_allclose(x_rh, x_escn, atol=1.0e-4)
    # endregion

    # region  grid_act to_grid
    display(Markdown(f"## Layer {layer_idx} grid_act to_grid"))

    print(
        {
            "rhescn": (
                x_rh := (
                    rearrange(
                        rhescn.activations[
                            f"LayerBlock_{layer_idx}.MessageBlock.act_rotate_inv.x_grid"
                        ][0],
                        "E (res_beta res_alpha) C -> E res_beta res_alpha C",
                        res_beta=10,
                    )
                )
            ),
            "escn": (
                x_escn := (
                    escn.activations[
                        f"LayerBlock_{layer_idx}.message_block.grid_act.x_grid"
                    ][0]
                )
            ),
        },
    )
    np.testing.assert_allclose(x_rh, x_escn, atol=1e-4)
    # endregion

    # region  grid_act silu
    display(Markdown(f"## Layer {layer_idx} grid_act silu"))

    print(
        {
            "rhescn": (
                x_rh := (
                    rearrange(
                        rhescn.activations[
                            f"LayerBlock_{layer_idx}.MessageBlock.act_rotate_inv.x_grid_silu"
                        ][0],
                        "E (res_beta res_alpha) C -> E res_beta res_alpha C",
                        res_beta=10,
                    )
                )
            ),
            "escn": (
                x_escn := (
                    escn.activations[
                        f"LayerBlock_{layer_idx}.message_block.grid_act.x_grid_silu"
                    ][0]
                )
            ),
        },
    )
    np.testing.assert_allclose(x_rh, x_escn, atol=1e-4)
    # endregion

    # region  grid_act to_sphere
    display(Markdown(f"## Layer {layer_idx} grid_act to_sphere"))

    print(
        {
            "rhescn": (
                x_rh := trimask_mmax(
                    rhescn.activations[
                        f"LayerBlock_{layer_idx}.MessageBlock.act_rotate_inv.x_sphere_silu"
                    ][0]
                )
            ),
            "escn": (
                x_escn := (
                    escn.activations[
                        f"LayerBlock_{layer_idx}.message_block.grid_act.x_sphere_silu"
                    ][0]
                )
            ),
        },
    )
    np.testing.assert_allclose(x_rh, x_escn, atol=1e-4)
    # endregion

    # region  rotate_inv
    display(Markdown(f"## Layer {layer_idx} rotate_inv"))

    print(
        {
            "rhescn": (
                x_rh := trimask(
                    rhescn.activations[
                        f"LayerBlock_{layer_idx}.MessageBlock.act_rotate_inv.x_sphere_rotated"
                    ][0]
                )
            ),
            "escn": (
                x_escn := (
                    escn.activations[
                        f"LayerBlock_{layer_idx}.message_block.x_target_rot_inv"
                    ][0]
                )
            ),
        },
    )
    np.testing.assert_allclose(x_rh, x_escn, atol=1e-4)
    # endregion

    # region  scatter
    display(Markdown(f"## Layer {layer_idx} scatter"))

    print(
        {
            "rhescn": (
                x_rh := trimask(
                    rhescn.activations[
                        f"LayerBlock_{layer_idx}.MessageBlock.x_scattered"
                    ][0]
                )
            ),
            "escn": (
                x_escn := (
                    escn.activations[
                        f"LayerBlock_{layer_idx}.message_block.x_target_reduce"
                    ][0]
                )
            ),
        },
    )
    np.testing.assert_allclose(x_rh, x_escn, atol=1e-4)
    # endregion

    # region  x_message
    display(Markdown(f"## Layer {layer_idx} x_message"))

    print(
        {
            "rhescn": (
                x_rh := trimask(
                    rhescn.activations[f"LayerBlock_{layer_idx}.x_message"][0]
                )
            ),
            "escn": (
                x_escn := (
                    escn.activations[
                        f"LayerBlock_{layer_idx}.message_block.x_target_reduce"
                    ][0]
                )
            ),
        },
    )
    np.testing.assert_allclose(x_rh, x_escn, atol=1e-4)
    # endregion

    # region  x_message after SO2
    display(Markdown(f"## Layer {layer_idx} x_message after SO2"))

    print(
        {
            "rhescn": (
                x_rh := trimask(
                    rhescn.activations[f"LayerBlock_{layer_idx}.x_message"][0]
                )
            ),
            "escn": (
                x_escn := escn.activations[f"LayerBlock_{layer_idx}.x_message"][0]
            ),
        },
    )
    np.testing.assert_allclose(x_rh, x_escn, atol=1e-4)
    # endregion

    # region  conv: to_grid_mat
    display(Markdown(f"## Layer {layer_idx} conv: to_grid_mat"))

    print(
        {
            "rhescn": (
                x_rh := trimask(
                    rhescn.activations[
                        f"LayerBlock_{layer_idx}.pointwise_grid_conv.to_grid_sh_tri"
                    ][0]
                )
            ),
            "escn": (
                x_escn := rearrange(
                    escn.activations[
                        f"LayerBlock_{layer_idx}.grid_conv.x_message_to_grid.to_grid_mat"
                    ][0],
                    "res_beta res_alpha l_sq -> (res_beta res_alpha) l_sq",
                )
            ),
        },
    )
    np.testing.assert_allclose(x_rh, x_escn, atol=1e-4)
    # endregion
    # region  conv: x_grid
    display(Markdown(f"## Layer {layer_idx} conv: x_grid"))

    print(
        {
            "rhescn": (
                x_rh := rhescn.activations[
                    f"LayerBlock_{layer_idx}.pointwise_grid_conv.s2_conv.x_grid"
                ][0]
            ),
            "escn": (
                x_escn := rearrange(
                    escn.activations[f"LayerBlock_{layer_idx}.grid_conv.x_grid"][0],
                    "N res_beta res_alpha C -> N (res_beta res_alpha) C",
                )
            ),
        },
    )
    np.testing.assert_allclose(x_rh, x_escn, atol=1e-4)
    # endregion

    # region  conv: x_grid_message
    display(Markdown(f"## Layer {layer_idx} conv: x_grid_message"))

    print(
        {
            "rhescn": (
                x_rh := rhescn.activations[
                    f"LayerBlock_{layer_idx}.pointwise_grid_conv.s2_conv.x_message_grid"
                ][0]
            ),
            "escn": (
                x_escn := rearrange(
                    escn.activations[
                        f"LayerBlock_{layer_idx}.grid_conv.x_grid_message"
                    ][0],
                    "N res_beta res_alpha C -> N (res_beta res_alpha) C",
                )
            ),
        },
    )
    np.testing.assert_allclose(x_rh, x_escn, atol=1e-4)
    # endregion

    # region  conv: x_grid_conv
    display(Markdown(f"## Layer {layer_idx} conv: x_grid_conv"))

    print(
        {
            "rhescn": (
                x_rh := rhescn.activations[
                    f"LayerBlock_{layer_idx}.pointwise_grid_conv.s2_conv.x_grid_conv"
                ][0]
            ),
            "escn": (
                x_escn := rearrange(
                    escn.activations[f"LayerBlock_{layer_idx}.grid_conv.x_grid_conv"][
                        0
                    ],
                    "N res_beta res_alpha C -> N (res_beta res_alpha) C",
                )
            ),
        },
    )
    np.testing.assert_allclose(x_rh, x_escn, atol=1e-4)
    # endregion

    # region  conv: x_message_from_grid
    display(Markdown(f"## Layer {layer_idx} conv: x_message_from_grid"))

    print(
        {
            "rhescn": (
                x_rh := trimask(
                    rhescn.activations[
                        f"LayerBlock_{layer_idx}.pointwise_grid_conv.x_message_updated"
                    ][0]
                )
            ),
            "escn": (
                x_escn := escn.activations[
                    f"LayerBlock_{layer_idx}.grid_conv.x_message_from_grid"
                ][0]
            ),
        },
    )
    np.testing.assert_allclose(x_rh, x_escn, atol=1e-4)
    # endregion

    # region  conv
    display(Markdown(f"## Layer {layer_idx} conv"))

    print(
        {
            "rhescn": (
                x_rh := trimask(
                    rhescn.activations[
                        f"LayerBlock_{layer_idx}.pointwise_grid_conv.x_message_updated"
                    ][0]
                )
            ),
            "escn": (
                x_escn := (
                    escn.activations[f"LayerBlock_{layer_idx}.x_message_updated"][0]
                )
            ),
        },
    )
    np.testing.assert_allclose(x_rh, x_escn, atol=1e-4)
    # endregion

In [6]:
for i in range(12):
    check_layer(i)

## Layer 0 x_edge

{
    'rhescn': array[2296, 128] f32 n=293888 (1.1Mb) x∈[-0.278, 4.464] μ=-0.008 σ=0.076,
    'escn': array[2296, 128] f32 n=293888 (1.1Mb) x∈[-0.278, 4.464] μ=-0.008 σ=0.076
}

## Layer 0 x_source

{
    'rhescn': array[2296, 25, 128] f32 n=7347200 (28Mb) x∈[-1.083, 1.133] μ=0.000 σ=0.069,
    'escn': array[2296, 25, 128] f32 n=7347200 (28Mb) x∈[-1.083, 1.133] μ=0.000 σ=0.069
}

## Layer 0 x_target

...

{
    'rhescn': array[107, 110, 128] f32 n=1506560 (5.7Mb) x∈[-30.557, 29.364] μ=0.096 σ=1.731,
    'escn': array[107, 110, 128] f32 n=1506560 (5.7Mb) x∈[-30.557, 29.364] μ=0.096 σ=1.731
}

## Layer 11 conv: x_message_from_grid

{
    'rhescn': array[107, 25, 128] f32 n=342400 (1.3Mb) x∈[-27.406, 25.823] μ=0.020 σ=1.156,
    'escn': array[107, 25, 128] f32 n=342400 (1.3Mb) x∈[-27.406, 25.823] μ=0.020 σ=1.156
}

## Layer 11 conv

{
    'rhescn': array[107, 25, 128] f32 n=342400 (1.3Mb) x∈[-27.406, 25.823] μ=0.020 σ=1.156,
    'escn': array[107, 25, 128] f32 n=342400 (1.3Mb) x∈[-27.406, 25.823] μ=0.020 σ=1.156
}

In [7]:
layer_idx = 1
display(Markdown(f"## Layer {layer_idx} rotate_inv"))


L = 4


def _select(x):
    range_ = slice((L) ** 2, (L + 1) ** 2)
    return x[:, range_]


print(
    {
        "rhescn": (
            x_rh := _select(
                trimask(
                    rhescn.activations[
                        f"LayerBlock_{layer_idx}.MessageBlock.act_rotate_inv.x_sphere_rotated"
                    ][0]
                )
            )
        ),
        "escn": (
            x_escn := _select(
                escn.activations[
                    f"LayerBlock_{layer_idx}.message_block.x_target_rot_inv"
                ][0]
            )
        ),
    },
)
np.testing.assert_allclose(x_rh, x_escn, atol=1e-5)

## Layer 1 rotate_inv

{
    'rhescn': array[2296, 9, 128] f32 n=2644992 (10Mb) x∈[-7.591, 6.900] μ=-0.001 σ=0.132,
    'escn': array[2296, 9, 128] f32 n=2644992 (10Mb) x∈[-7.591, 6.900] μ=-0.001 σ=0.132
}